# Cancellation Protection

## Imports

In [1]:
## Reference: https://cloud.google.com/bigquery/docs/bigquery-storage-python-pandas
## Docs: https://googleapis.dev/python/bigquery/latest/index.html
import pandas as pd
import google.auth
from google.cloud import bigquery
from google.cloud import bigquery_storage

## Makes reading more efficient (i think)
import google.cloud.bigquery.magics
google.cloud.bigquery.magics.context.use_bqstorage_api = True


# Explicitly create a credentials object. This allows you to use the same
# credentials for both the BigQuery and BigQuery Storage clients, avoiding
# unnecessary API calls to fetch duplicate authentication tokens.
credentials, your_project_id = google.auth.default(
    scopes=["https://www.googleapis.com/auth/cloud-platform"]
)

project_id = 'analytics-hub-prod'



In [2]:
# Make clients.
%load_ext google.cloud.bigquery

bqclient = bigquery.Client(credentials=credentials, project=your_project_id,)
bqstorageclient = bigquery_storage.BigQueryReadClient(credentials=credentials)
google.cloud.bigquery.magics.context.project = project_id

## Data

### Read in data

#### Base

In [7]:
%%bigquery base
SELECT 
    product_name
    ,customer_locator_id
    ,opportunity_locator_id
    ,customer_reference
    ,mid
    ,CAST(created_date AS DATE) AS created_date
    ,docs_in_date
    ,credit_approved_date
    ,CAST(live_date AS DATE) AS live_date
    ,DATE_DIFF(docs_in_date ,CAST(created_date AS DATE),DAY) AS created_to_docs_in
    ,DATE_DIFF(credit_approved_date ,docs_in_date, DAY) AS docs_in_to_ca
    ,DATE_DIFF(CAST(live_date AS DATE) ,credit_approved_date, DAY) AS ca_to_live
    ,is_live
    ,churn_date
    ,opp_owner
    ,opp_owner_manager
    ,CASE 
        WHEN opp_owner_manager IN ('David Knoyle', 'Jonathan Atkins', 'Matt Light', 'Targino Kalid-Filho') THEN 1
        WHEN opp_owner_manager IN ('Richard Sarsfield', 'Craig Calvert', 'James Coade', 'James White') THEN 0 
    END AS pilot_group
    ,postcode_area
    ,signed_cto
    ,signed_cto_group
    ,ltr_total
    ,atv
FROM `analytics-hub-dev.commercial.lk_fct_opps_all_products`
WHERE 
    created_date >= '2021-01-14'
    AND customer_type = 'SW'
    AND sales_channel_grouped = 'Field Sales'

In [8]:
%%bigquery cancellation_rebate
SELECT
    CASE WHEN cr.product_name = 'paymentsense' THEN o.mid ELSE l.mid END AS mid
    ,CASE WHEN cr.product_name = 'paymentsense' THEN o.customer_locator_id ELSE l.prospect_id END AS customer_locator_id
    ,CASE WHEN cr.product_name = 'paymentsense' THEN o.opportunity_locator_id ELSE l.quote_id END AS opportunity_locator_id
    ,l.customer_reference
    ,l.location_id
    ,cr.product_name
    ,MIN(cr.accounting_date) AS accounting_date
    ,SUM(cr.transaction_debit_amount) AS transaction_debit_amount
    ,SUM(cr.transaction_credit_amount) AS transaction_credit_amount
    ,SUM(cr.net_amount) AS net_amount
    ,SUM(cr.net_amount_per_mid) AS net_amount_per_mid
    ,MIN(cr.first_transacting_month) AS first_transacting_month
FROM `analytics-hub-prod.finance.dim_cancellation_rebates` cr
LEFT JOIN `analytics-hub-prod.paymentsense_boost.dim_locations` l ON cr.bank_mid = l.mid
LEFT JOIN `analytics-hub-dev.commercial.lk_fct_opps_all_products` o ON cr.bank_mid = o.mid
WHERE 
    accounting_date >= '2021-01-14'
GROUP BY 1,2,3,4,5,6

In [9]:
%%bigquery tickets
SELECT 
    mid
    ,count(distinct ticket_id) AS ticket_count
    ,count(distinct case when ticket_direction = 'Inbound' THEN ticket_id ELSE NULL END ) as inbound_ticket_count
    ,count(distinct case when operations_function = 'Complaints' THEN ticket_id ELSE NULL END ) as complaints_ticket_count
    ,count(distinct case when operations_function = 'Customer Support' THEN ticket_id ELSE NULL END ) as customer_support_ticket_count
    ,count(distinct case when operations_function = 'Retentions' THEN ticket_id ELSE NULL END ) as retentions_ticket_count
FROM `analytics-hub-prod.operations.fct_tickets__all`
GROUP BY 1

In [10]:
%%bigquery calls
SELECT 
    mid,
    count(distinct master_call_id) AS call_count
FROM `analytics-hub-prod.operations.fct_contacts`
GROUP BY 1